In [1]:
#import pandas
import pandas as pd
import numpy as np

In [2]:
#import sample data
data = pd.read_csv('../monthly-data/2020/01/patterns-part1.csv')
data['safegraph_place_id']
data.dtypes

safegraph_place_id            object
location_name                 object
street_address                object
city                          object
region                        object
postal_code                    int64
safegraph_brand_ids           object
brands                        object
date_range_start              object
date_range_end                object
raw_visit_counts               int64
raw_visitor_counts             int64
visits_by_day                 object
poi_cbg                      float64
visitor_home_cbgs             object
visitor_daytime_cbgs          object
visitor_work_cbgs             object
visitor_country_of_origin     object
distance_from_home           float64
median_dwell                 float64
bucketed_dwell_times          object
related_same_day_brand        object
related_same_month_brand      object
popularity_by_hour            object
popularity_by_day             object
device_type                   object
dtype: object

In [3]:
df = data

from ast import literal_eval
df.visitor_home_cbgs = df.visitor_home_cbgs.apply(literal_eval)

In [4]:
#pd.json_normalize(df.visitor_home_cbgs)

MemoryError: Unable to allocate 1.73 TiB for an array with shape (1078206, 219906) and data type object

In [ ]:
df2 = df.visitor_home_cbgs.apply(pd.Series)
combined_df = pd.concat([df, df2], axis=1)
combined_df = combined_df.drop(['visitor_home_cbgs'], axis=1)
combined_df

In [ ]:
# Get an array of the Transient CBGs for Bloomington and convert them each to strings
transient_df = pd.read_csv('bton_transient_cbgs.csv')
transient_df.census_block_group = transient_df.census_block_group.astype(str)
transient_df.dtypes

In [ ]:
transient = transient_df.census_block_group.to_numpy()
transient

In [ ]:
# Get array of dataframe columns
list(combined_df.columns[1:26])
visitor_cbgs = combined_df.columns[27:].to_numpy()
visitor_cbgs

In [ ]:
# Find out which CBGs appear in BOTH the bton_transient list as well as in the visit data
# This should give us a list of only locations that were visited by residents of the Bton transient CBGs
bton_transient_visits = np.intersect1d(transient, visitor_cbgs)
bton_transient_visits

In [ ]:
melted_df = pd.melt(combined_df, id_vars=['safegraph_place_id'], value_vars=bton_transient_visits, var_name='home_cbg', value_name='visits')
melted_df = melted_df.dropna()
melted_df.visits = melted_df.visits.astype(int)
melted_df

In [1]:
# Join this back with the original data so that each location has a separate row for each home_cbg in the Bton transient list

merged_df = pd.merge(data, melted_df, on='safegraph_place_id')
merged_df

NameError: name 'pd' is not defined